Base에서 실행

# Textcuboid


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## Read the data into a dataframe
import json
import pandas as pd
import numpy as np
import re
import string
import nltk


data=[]
with open('C:/Users/user/Desktop/bilm-tf-master/textdataset/News_Category_Dataset_v3.json', 'r') as f:
    for line in f:
        content = json.loads(line)
        data.append(content)
df = pd.DataFrame(data=data)

In [2]:
df = df[~df['short_description'].apply(lambda x: len(x)==0)]
print(df.shape)

#중복되는 카테고리 통합
df["category"] = df.category.replace(
    {
        "THE WORLDPOST": "WORLD NEWS",
        "WORLDPOST": "WORLD NEWS",
        "ARTS": "ARTS & CULTURE",
        "CULTURE & ARTS": "ARTS & CULTURE",
        "HEALTHY LIVING": "WELLNESS",
        "QUEER VOICES": "VOICES",
        "LATINO VOICES": "VOICES",
        "BLACK VOICES": "VOICES",
        "STYLE": "STYLE & BEATUY",
        "GREEN": "ENVIRONMENT",
        "TASTE": "FOOD & DRINK",
        "MONEY": "BUSINESS",
        "PARENTING": "PARENTS"
    }
)

df = df[['headline', 'short_description','category']]
df['input_data']= df.apply(lambda x: str(x['headline']) + str(x['short_description']), axis=1)

def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    #text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text

df['clean_text']=df['input_data'].apply(clean_text)

class0=list(df[df['category'] == 'POLITICS'].sample(1000, random_state=1)['clean_text'])
class1=list(df[df['category'] == 'WELLNESS'].sample(1000, random_state=1)['clean_text'])
class2=list(df[df['category'] == 'ENTERTAINMENT'].sample(1000, random_state=1)['clean_text'])
class3=list(df[df['category'] == 'PARENTS'].sample(1000, random_state=1)['clean_text'])
class4=list(df[df['category'] == 'VOICES'].sample(1000, random_state=1)['clean_text'])
class5=list(df[df['category'] == 'STYLE & BEAUTY'].sample(1000, random_state=1)['clean_text'])
class6=list(df[df['category'] == 'TRAVEL'].sample(1000, random_state=1)['clean_text'])
class7=list(df[df['category'] == 'FOOD & DRINK'].sample(1000, random_state=1)['clean_text'])
class8=list(df[df['category'] == 'WORLD NEWS'].sample(1000, random_state=1)['clean_text'])
class9=list(df[df['category'] == 'BUSINESS'].sample(1000, random_state=1)['clean_text'])

X_list=class0+class1+class2+class3+class4+class5+class6+class7+class8+class9

y=[]
for i in range(10):
    y+=[i]*1000

clr_x_data=[]
pattern = '[^a-z ]'
for sen in X_list:
    clr_x_data.append(re.sub(pattern, ' ', sen))

X_list=[]
for sen in clr_x_data:
    X_list.append(' '.join(sen.split()))

train_idx=[]
for i in range(10):
    train_idx+=[j+i*1000 for j in range(800)]

test_idx=[]
for i in range(10):
    test_idx+=[j+800+i*1000 for j in range(200)]

x_train=[]
y_train=[]
for i in train_idx:
    x_train.append(X_list[i])
    y_train.append(y[i])

x_test=[]
y_test=[]
for i in test_idx:
    x_test.append(X_list[i])
    y_test.append(y[i])

to_txt_filter=x_train+x_test
y=list(y_train)+list(y_test)

(189815, 6)


## 2) textcuboid 분류

In [3]:
textcuboid=np.load('./1-Channel textcuboid_nc(elmo).npy')

In [4]:
textcuboid_test=np.load('./1-Channel textcuboid_test_nc(elmo).npy')

In [5]:
tmp = [[x,y] for x, y in zip(textcuboid, y_train)]
import random
random.shuffle(tmp)
textcuboid = [n[0] for n in tmp]
y_train = [n[1] for n in tmp]
textcuboid=np.array(textcuboid)
y_train=np.array(y_train)

In [6]:
import keras
from tensorflow.keras import layers

In [34]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, verbose=1),
]

In [35]:
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model


input_layer = Input(shape=(68, 256))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(input_layer)
pooling = GlobalMaxPooling1D()(conv1)

x = Dense(1024, activation='relu')(pooling)
x = Dropout(0.5)(x)
output_layer = Dense(10, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)



In [36]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [37]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 68, 256)]         0         
                                                                 
 conv1d_3 (Conv1D)           (None, 68, 1024)          263168    
                                                                 
 global_max_pooling1d_3 (Glo  (None, 1024)             0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 1024)              1049600   
                                                                 
 dropout_3 (Dropout)         (None, 1024)              0         
                                                                 
 dense_7 (Dense)             (None, 10)                10250     
                                                           

In [38]:
x_train=textcuboid[1000:]
x_val=textcuboid[:1000]
y_train1=y_train[1000:]
y_val=y_train[:1000]

In [39]:
history = model.fit(x_train, y_train1,callbacks=callbacks, epochs=20,batch_size=256,validation_data=(x_val, y_val))

Epoch 1/20
28/28 [==============================] - 1s 15ms/step - loss: 1.7830 - accuracy: 0.4359 - val_loss: 1.0024 - val_accuracy: 0.6570 - lr: 0.0010
Epoch 2/20
28/28 [==============================] - 0s 10ms/step - loss: 0.9238 - accuracy: 0.6959 - val_loss: 0.8508 - val_accuracy: 0.7190 - lr: 0.0010
Epoch 3/20
28/28 [==============================] - 0s 10ms/step - loss: 0.7759 - accuracy: 0.7500 - val_loss: 0.7915 - val_accuracy: 0.7420 - lr: 0.0010
Epoch 4/20
28/28 [==============================] - 0s 10ms/step - loss: 0.6586 - accuracy: 0.7806 - val_loss: 0.7803 - val_accuracy: 0.7380 - lr: 0.0010
Epoch 5/20
28/28 [==============================] - 0s 9ms/step - loss: 0.5644 - accuracy: 0.8137 - val_loss: 0.8045 - val_accuracy: 0.7290 - lr: 0.0010
Epoch 6/20
28/28 [==============================] - 0s 9ms/step - loss: 0.4764 - accuracy: 0.8469 - val_loss: 0.8204 - val_accuracy: 0.7340 - lr: 0.0010
Epoch 7/20
28/28 [==============================] - 0s 9ms/step - loss: 0.3795

In [40]:
y_test=np.array(y_test)

In [41]:
model.evaluate(textcuboid_test,y_test)

63/63 [==============================] - 0s 2ms/step - loss: 0.8677 - accuracy: 0.7430


[0.8676668405532837, 0.7429999709129333]

In [42]:
from keras.models import load_model
model = load_model('best_model.h5')
model.evaluate(textcuboid_test,y_test)

63/63 [==============================] - 0s 2ms/step - loss: 0.8016 - accuracy: 0.7465


[0.8016155362129211, 0.7465000152587891]

In [43]:
np.average([74.7,74.9,74.7])

74.76666666666667